In [8]:
import re
import numpy as np
import pandas as pd
import os
from scipy import linalg, spatial
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import (CountVectorizer, TfidfTransformer, TfidfVectorizer)
from sklearn.cluster import KMeans


from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [9]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hsroc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hsroc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
corpus = [

          "With all of the critical success Downey had experienced throughout his career, he had not appeared in a blockbuster film. That changed in 2008 when Downey starred in two critically and commercially successful films, Iron Man and Tropic Thunder. In the article Ben Stiller wrote for Downey's entry in the 2008 edition of The Time 100, he offered an observation on Downey's commercially successful summer at the box office.",
          "On June 14, 2010, Downey and his wife Susan opened their own production company called Team Downey. Their first project was The Judge.",
          "Robert John Downey Jr. is an American actor, producer, and singer. His career has been characterized by critical and popular success in his youth, followed by a period of substance abuse and legal troubles, before a resurgence of commercial success in middle age.",
          "In 2008, Downey was named by Time magazine among the 100 most influential people in the world, and from 2013 to 2015, he was listed by Forbes as Hollywood's highest-paid actor. His films have grossed over $14.4 billion worldwide, making him the second highest-grossing box-office star of all time.",
          "Wanderer, look within and bless yourself. If you have never experienced this harmonizing inherent in nature, it can be difficult to grow. It can be difficult to know where to begin"
          
          ]

stop_words = set(stopwords.words('english')) | set(stopwords.words('portuguese'))

In [71]:
corpus_lero_lero = [

        "Residence certainly elsewhere something she preferred cordially law. Age his surprise formerly mrs perceive few stanhill moderate. Of in power match on truth worse voice would. Large an it sense shall an match learn. By expect it result silent in formal of. Ask eat questions abilities described elsewhere assurance. Appetite in unlocked advanced breeding position concerns as. Cheerful get shutters yet for repeated screened. An no am cause hopes at three. Prevent behaved fertile he is mistake on. ",
        "As collected deficient objection by it discovery sincerity curiosity. Quiet decay who round three world whole has mrs man. Built the china there tried jokes which gay why. Assure in adieus wicket it is. But spoke round point and one joy. Offending her moonlight men sweetness see unwilling. Often of it tears whole oh balls share an. ",
        "In as name to here them deny wise this. As rapid woody my he me which. Men but they fail shew just wish next put. Led all visitor musical calling nor her. Within coming figure sex things are. Pretended concluded did repulsive education smallness yet yet described. Had country man his pressed shewing. No gate dare rose he. Eyes year if miss he as upon. ",
        "Arrived totally in as between private. Favour of so as on pretty though elinor direct. Reasonable estimating be alteration we themselves entreaties me of reasonably. Direct wished so be expect polite valley. Whose asked stand it sense no spoil to. Prudent you too his conduct feeling limited and. Side he lose paid as hope so face upon be. Goodness did suitable learning put.",
        "Tolerably earnestly middleton extremely distrusts she boy now not. Add and offered prepare how cordial two promise. Greatly who affixed suppose but enquire compact prepare all put. Added forth chief trees but rooms think may. Wicket do manner others seemed enable rather in. Excellent own discovery unfeeling sweetness questions the gentleman. Chapter shyness matters mr parlors if mention thought.",
        "Debating me breeding be answered an he. Spoil event was words her off cause any. Tears woman which no is world miles woody. Wished be do mutual except in effect answer. Had boisterous friendship thoroughly cultivated son imprudence connection. Windows because concern sex its. Law allow saved views hills day ten. Examine waiting his evening day passage proceed. ",
        "Ought these are balls place mrs their times add she. Taken no great widow spoke of it small. Genius use except son esteem merely her limits. Sons park by do make on. It do oh cottage offered cottage in written. Especially of dissimilar up attachment themselves by interested boisterous. Linen mrs seems men table. Jennings dashwood to quitting marriage bachelor in. On as conviction in of appearance apartments boisterous.",
        "Who are we? Where on the great journey will we be reborn? Humankind has nothing to lose. Reality has always been overflowing with starseeds whose chakras are immersed in starfire."
        "Full age sex set feel her told. Tastes giving in passed direct me valley as supply. End great stood boy noisy often way taken short. Rent the size our more door. Years no place abode in ﻿no child my. Man pianoforte too solicitude friendship devonshire ten ask. Course sooner its silent but formal she led. Extensive he assurance extremity at breakfast. Dear sure ye sold fine sell on. Projection at up connection literature insensible motionless projecting. ",
        "Had repulsive dashwoods suspicion sincerity but advantage now him. Remark easily garret nor nay. Civil those mrs enjoy shy fat merry. You greatest jointure saw horrible. He private he on be imagine suppose. Fertile beloved evident through no service elderly is. Blind there if every no so at. Own neglected you preferred way sincerity delivered his attempted. To of message cottage windows do besides against uncivil. ",
        "Affronting imprudence do he he everything. Sex lasted dinner wanted indeed wished out law. Far advanced settling say finished raillery. Offered chiefly farther of my no colonel shyness. Such on help ye some door if in. Laughter proposal laughing any son law consider. Needed except up piqued an. ",
        "Considered an invitation do introduced sufficient understood instrument it. Of decisively friendship in as collecting at. No affixed be husband ye females brother garrets proceed. Least child who seven happy yet balls young. Discovery sweetness principle discourse shameless bed one excellent. Sentiments of surrounded friendship dispatched connection is he. Me or produce besides hastily up as pleased. Bore less when had and john shed hope. ",
        "Received overcame oh sensible so at an. Formed do change merely to county it. Am separate contempt domestic to to oh. On relation my so addition branched. Put hearing cottage she norland letters equally prepare too. Replied exposed savings he no viewing as up. Soon body add him hill. No father living really people estate if. Mistake do produce beloved demesne if am pursuit. ",
        "Its sometimes her behaviour are contented. Do listening am eagerness oh objection collected. Together gay feelings continue juvenile had off one. Unknown may service subject her letters one bed. Child years noise ye in forty. Loud in this in both hold. My entrance me is disposal bachelor remember relation. ",
        "Mr do raising article general norland my hastily. Its companions say uncommonly pianoforte favourable. Education affection consulted by mr attending he therefore on forfeited. High way more far feet kind evil play led. Sometimes furnished collected add for resources attention. Norland an by minuter enquire it general on towards forming. Adapted mrs totally company two yet conduct men. ",
        "Extremity direction existence as dashwoods do up. Securing marianne led welcomed offended but offering six raptures. Conveying concluded newspaper rapturous oh at. Two indeed suffer saw beyond far former mrs remain. Occasional continuing possession we insensible an sentiments as is. Law but reasonably motionless principles she. Has six worse downs far blush rooms above stood. ",
        "To sure calm much most long me mean. Able rent long in do we. Uncommonly no it announcing melancholy an in. Mirth learn it he given. Secure shy favour length all twenty denote. He felicity no an at packages answered opinions juvenile. ",
        "Am finished rejoiced drawings so he elegance. Set lose dear upon had two its what seen. Held she sir how know what such whom. Esteem put uneasy set piqued son depend her others. Two dear held mrs feet view her old fine. Bore can led than how has rank. Discovery any extensive has commanded direction. Short at front which blind as. Ye as procuring unwilling principle by. ",
    
    
          ]


In [72]:
def filtering(corpus):
    filtered_document= []
    filtered_text = []

    for document in corpus:

        clean_document = " ".join(re.sub(r"[^A-Za-z \—]+", " ", document).split())

        document_tokens = word_tokenize(clean_document)

        for word in document_tokens:
            if word not in stop_words:
                filtered_document.append(word)

        filtered_text.append(' '.join(filtered_document))
    return filtered_text

In [73]:
def generate_document_term_matrix(text, corpus):
    vectorizer = CountVectorizer()
    document_n = []
    n = 1
    for i in corpus:
        document_n.append("Document {n}")
        n+=1
    counts_matrix = vectorizer.fit_transform(text)

    feature_names = vectorizer.get_feature_names()

    count_matrix_df = pd.DataFrame(counts_matrix.toarray(), columns=feature_names)
    count_matrix_df.index = document_n

    return np.array(count_matrix_df)

In [74]:
generate_document_term_matrix(filtering(corpus), corpus)


array([[0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 2, 0, 1, 1,
        0, 4, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1, 0, 0, 1, 1, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 1,
        0, 6, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 2, 1,
        1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0],
       [1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1,
        0, 7, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
        0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 3, 2, 1, 1, 1, 1, 2, 1,
        1,

In [15]:
def reduce_dimensionality(Sigma, VT, k):
    indexes = []
    for index in range(len(Sigma)):
        indexes.append(index)
    sorted_sigma, indexes = (list(t) for t in zip(*sorted(zip(Sigma, indexes))))
    culling = range(len(Sigma) - k)
    Sigma = np.delete(Sigma, indexes[0:len(culling)], 0)
    VT = np.delete(VT, indexes[0:len(culling)], 1)
    return Sigma, VT

U, Sigma, VT = linalg.svd(generate_document_term_matrix(filtering(corpus), corpus))

In [22]:
def generate_absolute_coherence_score(Sigma_Array, VT, n, m):
    dn = VT[:,n]
    dm = VT[:,m]
    vecn = [Sigma_Array[i]*dn[i] for i in range(len(Sigma_Array))]
    vecm = [Sigma_Array[i]*dm[i] for i in range(len(Sigma_Array))]
    vecn = np.array(vecn)
    vecm = np.array(vecm)
    coherence_score = 1 - spatial.distance.cosine(vecn, vecm)
    return coherence_score

In [23]:
def avg_coherence(corpus, k):
    U, Sigma, VT = linalg.svd(generate_document_term_matrix(filtering(corpus), corpus))
    Sigma, VT = reduce_dimensionality(Sigma, VT, k)
    coherence = 0
    for n in range(len(corpus)-1):
        m = n + 1
        coherence_score = generate_absolute_coherence_score(Sigma, VT, n, m)
        coherence = coherence + coherence_score
    coherence = coherence/len(corpus)
    return coherence

In [90]:
corpus_coherence = avg_coherence(corpus_lero_lero, 5)
print(corpus_coherence)
# k-number of dimensions of array is a hyperparameter that must be configured to ensure distinction between incoherent and coherent sentences. high-k: overfitting. low k: underfitting

0.6763653126334445


In [82]:
def article_lsa_implement(source, k):
    n = 0
    avg_avg_coh = 0
    folder = source + "_articles"
    for file in os.listdir(folder):
        print(file)
        
        openfile = open(folder + "/" + file, "r")
        paragraphlist = openfile.readlines()
        coh = 0
        try:
            coh = avg_coherence(paragraphlist, k)
            n+=1
        except:
            print(paragraphlist)
        avg_avg_coh = avg_avg_coh + coh
    avg_avg_coh = avg_avg_coh/n
    return avg_avg_coh
def posts_lsa_implement(source, k):
    n = 0
    avg_avg_coh = 0
    folder = source + "_posts"
    for file in os.listdir(folder):
        openfile = open(folder + "/" + file, "r")
        paragraphlist = openfile.readlines()
        if len(paragraphlist) >1:
            n+=1
            print(file)
            try:
                coh = avg_coherence(paragraphlist, k)
            except:
                print("falha")
            avg_avg_coh = avg_avg_coh + coh
    avg_avg_coh = avg_avg_coh/n
    print(n)
    return avg_avg_coh

In [89]:
print(article_lsa_implement("", 5))

bbc_article1.txt
bbc_article10.txt
bbc_article11.txt
[]
bbc_article12.txt
bbc_article13.txt
bbc_article14.txt
bbc_article15.txt
bbc_article16.txt
bbc_article17.txt
bbc_article18.txt
bbc_article2.txt
bbc_article3.txt
bbc_article4.txt
bbc_article5.txt
bbc_article6.txt
bbc_article7.txt
bbc_article8.txt
bbc_article9.txt
0.7482454328251573


In [84]:
print(posts_lsa_implement("wsb", 10))

wsb_post1.txt
wsb_post15.txt
wsb_post19.txt
wsb_post2.txt
wsb_post28.txt
wsb_post3.txt
wsb_post42.txt
wsb_post49.txt
wsb_post50.txt
wsb_post51.txt
wsb_post54.txt
wsb_post7.txt
12
0.5793210725903557
